In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!pip uninstall -y torch torchvision torchaudio triton nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 nvidia-nccl-cu12

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
Found existing installation: nvidia-cublas-cu12 12.5.3.2
Uninstalling nvidia-cublas-cu12-12.5.3.2:
  Successfully uninstalled nvidia-cublas-cu12-12.5.3.2
Found existing installation: nvidia-cuda-runtime-cu12 12.5.82
Uninstalling nvidia-cuda-runtime-cu12-12.5.82:
  Successfully uninstalled nvidia-cuda-runtime-cu12-12.5.82
Found existing installation: nvidia-cudnn-cu12 9.3.0.75
Uninstalling nvidia-cudnn-cu12-9.3.0.75:
  Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
Found exi

In [3]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu125

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu125
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -q datasets bitsandbytes accelerate onnxruntime evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [5]:
!pip install pydub

In [6]:
import torch
import triton
import torch.backends.cudnn as cudnn

print("Torch Version:", torch.__version__)
print("Torch CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())
print("Triton Version:", triton.__version__)
print("cuDNN Version:", cudnn.version())

Torch Version: 2.6.0+cu124
Torch CUDA Version: 12.4
CUDA Available: True
Triton Version: 3.2.0
cuDNN Version: 90100


In [7]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from google.colab import drive
import time
import warnings
import numpy as np
import onnxruntime
from datasets import load_dataset
from itertools import islice
from torch.utils.data import IterableDataset
import json
import os
import wave
import threading
import textwrap
from IPython.display import clear_output
import evaluate
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
import collections
import pickle
import random
from pydub import AudioSegment

In [8]:
fnumbers = ['3148-2','3147-1','conf_2690_2690_00862467','conf_2688_2688_00862561']
wav_fnames = []
for f in fnumbers:
    for i in range(1,9):
        wav_fnames.append(f + f'_{i}.wav')

In [9]:
wav_fnames

['3148-2_1.wav',
 '3148-2_2.wav',
 '3148-2_3.wav',
 '3148-2_4.wav',
 '3148-2_5.wav',
 '3148-2_6.wav',
 '3148-2_7.wav',
 '3148-2_8.wav',
 '3147-1_1.wav',
 '3147-1_2.wav',
 '3147-1_3.wav',
 '3147-1_4.wav',
 '3147-1_5.wav',
 '3147-1_6.wav',
 '3147-1_7.wav',
 '3147-1_8.wav',
 'conf_2690_2690_00862467_1.wav',
 'conf_2690_2690_00862467_2.wav',
 'conf_2690_2690_00862467_3.wav',
 'conf_2690_2690_00862467_4.wav',
 'conf_2690_2690_00862467_5.wav',
 'conf_2690_2690_00862467_6.wav',
 'conf_2690_2690_00862467_7.wav',
 'conf_2690_2690_00862467_8.wav',
 'conf_2688_2688_00862561_1.wav',
 'conf_2688_2688_00862561_2.wav',
 'conf_2688_2688_00862561_3.wav',
 'conf_2688_2688_00862561_4.wav',
 'conf_2688_2688_00862561_5.wav',
 'conf_2688_2688_00862561_6.wav',
 'conf_2688_2688_00862561_7.wav',
 'conf_2688_2688_00862561_8.wav']

In [10]:
dataset_repo_test = "johnlohjy/imda_nsc_p3_test_noiseaugmented_mini"
dataset_test = load_dataset(dataset_repo_test, split='test', streaming=True, trust_remote_code=True)

imda_nsc_p3_test_noiseaugmented_mini.py:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

In [11]:
RATE = 16000
chunk = 4096
no_voice_activity_threshold = 20
references = []
total_audio_len = 0
total_audio_len_extra_silences = 0


#####################CREATE CONTINOUS AUDIO ARRAY ONCE S.T ITS REPRODUCIBLE############
# Store all audio samples
all_audio_samples = []

for sample in dataset_test:
  # If its a sample we want
  if sample['path'].split('/')[1] in wav_fnames:
    # Get a single audio samples data and convert from float 64 to float 32
    # Append it to all_audio_samples
    audio_sample = sample["audio"]["array"].astype(np.float32)
    all_audio_samples.append(audio_sample)

    # Increment the audio length
    total_audio_len += len(audio_sample)/16000
    total_audio_len_extra_silences += len(audio_sample)/16000

    # Insert random lengths of silence (1s to 5s) at the end of the curr sample
    silence = np.zeros(random.randint(1, 5)*RATE, dtype=np.float32)
    all_audio_samples.append(silence)

    # Increment the total_audio_len_extra_silences with silence
    total_audio_len_extra_silences += len(silence)/16000

    # Add the reference
    references.append(sample["sentence"])

# Concatenate all audio samples (with silences)
audio_samples = np.concatenate(all_audio_samples)

# Add no_voice_activity_threshold chunks of silence to the concatenated audio samples so the lastest segment of audio will be saved
silence = np.zeros(chunk*(no_voice_activity_threshold+5), dtype=np.float32)
audio_samples = np.concatenate((audio_samples, silence))
total_audio_len_extra_silences += len(silence)/16000
#####################CREATE CONTINOUS AUDIO ARRAY ONCE S.T ITS REPRODUCIBLE############

In [12]:
######################SAVE CONTINOUS ARRAYS######################
with open(f"test_audio_{total_audio_len/60}_mins.pkl", "wb") as f:
    pickle.dump(audio_samples, f)

with open(f"test_references.pkl", "wb") as f:
    pickle.dump(references, f)

with open(f"test_wavfnames.pkl", "wb") as f:
    pickle.dump(wav_fnames, f)
######################SAVE CONTINOUS ARRAYS######################

In [13]:
print(f'The total audio length is {total_audio_len}')
print(f'The total audio length with silences is {total_audio_len_extra_silences}')

The total audio length is 825.1258124999998
The total audio length with silences is 916.5258124999998


In [19]:
snr_wavfile_values = {}

with open("data_test_waves_snr.txt", "r") as file:
    for line in file:
        key, value = line.strip().split(' ')
        snr_wavfile_values[key] = value

snr_wavfile_counts = collections.defaultdict(int)

for fname in wav_fnames:
    snr_wavfile_counts[snr_wavfile_values[fname.split('.')[0]]] += 1

In [20]:
snr_wavfile_counts

defaultdict(int, {'10': 10, 'none': 6, '5': 11, '15': 5})

In [21]:
with open(f"test_snr_distribution.pkl", "wb") as f:
    pickle.dump(snr_wavfile_counts, f)

In [22]:
total_audio_len/60

13.752096874999998

In [23]:
916.5258124999998/60

15.27543020833333